In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Travelling Salesman Problem/Large/'

file_names = ['berlin52.npz', 'brazil58.npz', 'st70.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Minimum Lazy', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 20

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=8)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=1)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=4000)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,54905,15308,0,15308
1,3249,285551,50784,0,50784
2,4761,5028,1340,0,1340
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,54905,30880,0,30880
1,3249,285551,132420,0,132420
2,4761,5028,3257,0,3257
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,2601,54905,10860,0,10860
1,3249,285551,50553,0,50553


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,Energy (minimisation) 10,Energy (minimisation) 11,Energy (minimisation) 12,Energy (minimisation) 13,Energy (minimisation) 14,Energy (minimisation) 15,Energy (minimisation) 16,Energy (minimisation) 17,Energy (minimisation) 18,Energy (minimisation) 19
0,10860,11053,10652,11681,10660,10845,10361,11023,11795,10735,11079,10627,11117,11240,10551,10982,11019,10317,11065,11287
1,50553,42064,45228,40715,43471,45980,44264,42728,41732,47544,44720,42596,40671,40306,44888,43938,46043,47276,40073,42994
2,1478,1329,1564,1506,1460,1517,1450,1605,1362,1405,1334,1387,1411,1306,1386,1357,1695,1553,1292,1294


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,Broken Constraints 10,Broken Constraints 11,Broken Constraints 12,Broken Constraints 13,Broken Constraints 14,Broken Constraints 15,Broken Constraints 16,Broken Constraints 17,Broken Constraints 18,Broken Constraints 19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,Feasible 10,Feasible 11,Feasible 12,Feasible 13,Feasible 14,Feasible 15,Feasible 16,Feasible 17,Feasible 18,Feasible 19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,20.0,1.0,14630.200000,505.056912
1,20.0,1.0,53332.650000,2652.882112
2,20.0,1.0,1454.250000,86.526464
Total,60.0,3.0,69417.100000,3244.465488
Mean,20.0,1.0,23139.033333,1081.488496
SD,0.0,0.0,26965.575836,1376.862505
,Feasible,Feasibility rate,Energy mean,Energy SD
0,20.0,1.0,30021.800000,1444.745820
1,20.0,1.0,121410.500000,5671.509894
2,20.0,1.0,3549.150000,270.815780


# Save results

In [13]:
data_folder = '../Data/Produced/Travelling Salesman Problem/Large/'
broken_constraints_greedy.to_pickle(data_folder + 'lazy_tspL_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'lazy_tspL_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'lazy_tspL_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'lazy_tspL_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'lazy_tspL_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'lazy_tspL_tabu_feasible.pkl')